In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [6]:
data = spark.read.csv('4.1 Ecommerce_Customers.csv.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



### Setting up DataFrame for Machine Learning

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
### Just considering the columns which contains continous values for the regression problem

In [11]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'], outputCol='features')

In [12]:
### Transforming the data
output = assembler.transform(data)

In [13]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
final_data = output.select('features', 'Yearly Amount Spent')

In [15]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [16]:
### Splitting the data into training and testing data
train_data, test_data = final_data.randomSplit([0.70, 0.30])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                347|
|   mean| 504.44243622060236|
| stddev|  77.78524717411285|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                153|
|   mean|  487.6829657565118|
| stddev|  81.74952151567079|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [21]:
### Creating our Linear Regression Model

In [22]:
lr  = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
lr_model = lr.fit(train_data)

In [24]:
test_results = lr_model.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.962323099432865|
|-17.434497280402297|
| -3.831681944939362|
|  7.254470764100688|
|  4.492635169264645|
|-20.981058667364437|
| 0.5828339503398183|
| -9.021281999893688|
| -4.784926533574037|
|  6.815128053513888|
|-0.9113404811961345|
|-26.067345220287393|
|-2.0763222629129245|
| -5.791790911751775|
| -3.612228157165191|
|-17.351747040187377|
|  8.451287282544683|
|-2.2610203675619687|
|-16.911526773202752|
| 12.127941593242326|
+-------------------+
only showing top 20 rows



In [26]:
test_results.rootMeanSquaredError

10.322843524602897

In [27]:
test_results.r2

0.9839499609770904